In [1]:
import os
import json
import hashlib
from dateutil import parser
from datetime import datetime

## cleanup erpnext

In [2]:
# cleanup projects
# cleanup task
# cleanup timesheet
# cleanup timelogs

In [51]:
from util.frappyclient import FrappeClient

In [53]:
%env ACERP_ADMINISTRATOR=Administrator
%env ACERP_ADMINISTRATOR_PASS=Pandion123456@

env: ACERP_ADMINISTRATOR=Administrator
env: ACERP_ADMINISTRATOR_PASS=Pandion123456@


In [54]:
conn = FrappeClient("https://acerp.pandion.vn")
conn.login(os.environ['ACERP_ADMINISTRATOR'], os.environ['ACERP_ADMINISTRATOR_PASS'])

{'message': 'Logged In', 'home_page': '/app', 'full_name': 'Administrator'}

In [5]:
# cleanup projects

In [56]:
conn.get_doc('Task', '0c1a559618')

{'name': '0c1a559618',
 'owner': 'info@acons.vn',
 'creation': '2023-12-20 15:12:24.036477',
 'modified': '2024-05-22 14:35:24.000299',
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'task_number': '3',
 'subject': 'Design Development',
 'type': 'Type1',
 'is_group': 1,
 'is_template': 1,
 'status': 'Template',
 'priority': 'Medium',
 'task_weight': 0.0,
 'expected_time': 0.0,
 'start': 0,
 'progress': 0.0,
 'duration': 0,
 'is_milestone': 0,
 'depends_on_tasks': '',
 'actual_time': 0.0,
 'total_costing_amount': 0.0,
 'total_expense_claim': 0.0,
 'total_billing_amount': 0.0,
 'company': 'ACONS',
 'lft': 15,
 'rgt': 16,
 'old_parent': '',
 'doctype': 'Task',
 'depends_on': []}

In [4]:
doc = conn.get_doc('Timesheet', 'TS-04-04-24-00293_HR-EMP-00007')

In [144]:
# cleanup timesheets
ts = conn.get_list('Timesheet', 
                   fields = ['name', 'doctype', 'status', 'creation'],
                   filters = [['creation', '>=', '2024-01-01']], limit_page_length=1000)
# ts = conn.get_list('Timesheet', fields = ['name', 'creation'])
# ts = conn.get_list('Timesheet', filters = {'creation': {'=': '2024-01-31 12:10:29.497799'}})

In [145]:
len(ts)

0

In [146]:
ts[0:10]

[]

In [143]:
# delete NO RUN
for t in ts:
    if t['status'] == 'Draft' or t['status'] == 'Cancelled':
        conn.delete('Timesheet', t['name'])
    elif t['status'] == 'Submitted':
        t['doctype'] = 'Timesheet'
        t['status'] = 'Cancelled'
        t['docstatus'] = 2
        conn.update(t)
        # conn.set_value('Timesheet', t['name'], 'status', 'Cancelled')
        conn.delete('Timesheet', t['name'])

In [ ]:
# cleanup tasks

In [21]:
tsk = conn.get_list('Task', 
                    fields = ['name', 'status', 'project', 'creation', 'is_group'],
                    filters = [['creation', '>=', '2024-01-01'], ['is_group', '=', '1']], limit_page_length=300)

In [22]:
len(tsk)

0

In [19]:
# tsk[0:10]

In [20]:
# delete NO RUN
for t in tsk:
    print(t['name'])
    t['doctype'] = 'Task'
    t['depends_on'] = []
    conn.update(t)
    tsk_child = conn.get_list('Task', 
                              fields = ['name', 'status', 'project', 'creation', 'is_group'],
                              filters = [['parent_task', '=', t['name']]], limit_page_length=1000)
    for tt in tsk_child:
        tt['doctype'] = 'Task'
        tt['parent_task'] = ''
        conn.update(tt)
        conn.delete('Task', tt['name'])
    # print(t['name'])
    conn.delete('Task', t['name'])

eebe8b4cf6
eedff68d84
ef8b5cc518
efa1419452
efa8742421
f0322607b4
f0948c7183
f120cff99d
f1b82a61f7
f23abb0c82
f377b82dfb
f3a5bb1db3
f3b431416b
f42362308c
f433d06c3e
f49760f142
f539f1feb7
f57fc1079d
f5a79b376a
f5cdd275ef
f756508623
f75d90bcda
f7bb222db3
f7d0f3532e
f871fb7f2b
fa184b6f8e
fa4f42459a
fa577361f7
faf91d8c15
fb2d1c48e7
fb6ef2782d
fbd6c4a9d8
fc22670c46
fccfc74c7b
fd925479b0
fe3d3e1cac
fedcde79bc
ff04316de8
ff5e940552
ffd2f3ad8d


In [23]:
tsk = conn.get_list('Task', 
                    fields = ['name', 'status', 'project', 'creation', 'is_group'],
                    filters = [['creation', '>=', '2024-01-01']], limit_page_length=300)

In [24]:
len(tsk)

34

In [25]:
# delete NO RUN
for tt in tsk:
    tt['doctype'] = 'Task'
    tt['parent_task'] = ''
    conn.update(tt)
    conn.delete('Task', tt['name'])

## 

In [217]:
# tsk[1]

In [220]:
# for t in tsk:
#     conn.delete('Task', t['name'])

In [218]:
# conn.delete('Task', 'e38sn3pg90')

In [205]:
# for t in tsk:
#     print(t['name'])
#     conn.delete('Task', t['name'])

In [188]:
# tsk[0]['doctype'] = 'Task'
# tsk[0]['depends_on'] = []
# conn.update(tsk[0])

In [206]:
# conn.get_doc('Task', 'e38p1tg5gg')

In [190]:
# conn.delete('Task', 'e38p1tg5gg')

In [165]:
# conn.get_doc('Task', 'lia3u8u03v')

In [161]:
# for t in tsk:
#     conn.delete('Task', t['name'])

### timesheet import with google sheet

#### process flow

In [1]:
# handle sheet file
    # for each row of sheets
        # handle row data
            # if row is empty
                # skip this row
            # if col_B == "P"
                # handle as parent task
                # if col_A is not empty (parent task is created)
                    # handle update parent task if any changes (compare hash)
                # else
                    # handle as new parent task
            # else
                # handle as task with timesheet
                # if col_A is not empty (timesheet is created)
                    # handle update timesheet if any changes (compare hash)
                # else
                    # handle as new timesheet
    # endfor
    # handle update col_A data of sheets
        # for each row of col_A array above
            # update batch of col_A in sheet
    # end of function

#### problems

In [ ]:
# 1. col_A: can be duplicate (user copy and paste with new row)
# 2. col_A: 
# 3.
# 4.

In [2]:
from util.google_sheet import GGSheet

In [3]:
EXCEL_TASK_PRIORITY = {
    "": "Medium",
    "1_Urgen": "Urgent",
    "2_Important": "High",
    "3_Medium": "Medium",
    "4_Low": "Low",
}
EXCEL_TYPE_PARENT_TASK = {
    "": "Other",
    "0_Pre CO": "Pre Concept",
    "1_CO": "Concept",
    "2_BD": "Basic Design",
    "3_DD": "Design Development",
    "4_TD": "Tender Doc",
    "5_CD": "Construction",
    "6_AU": "Authorship",
    "7_Other": "Other",
}
EXCEL_TASK_STATUS = {
    "": "Open",
    "1_Open": "Open",
    "2_In progress": "In Progress",
    "3_Pending": "Pending",
    "4_Cancel": "Cancel",
    "5_Done": "Done",
    "6_Review": "Review",
}
EXCEL_TIME_SHEET_STATUS = {
    "": "Draft",
    "Open": "Draft",
    "In Progress": "Draft",
    "Pending": "Draft",
    "Review": "Draft",
    "Done": "Submitted",
    "Cancel": "Cancelled",
}
EXCEL_TIME_SHEET_DOC_STATUS = {
    "": 0,
    "Draft": 0,
    "Submitted": 1,
    "Cancelled": 2,
}

In [4]:
row_of_date = 3
row_start   = 5
row_end     = 1200
company     = "ACONS"

In [5]:
# url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
# worksheet_name="BAP"

In [6]:
# url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
# worksheet_name="T2"

In [7]:
# ggSheet = GGSheet(url_file, worksheet_name)

In [73]:
def mapping_row_with_attr(row_data):
    attrs = {}
    attrs["import_key"]          = row_data["A"] if "A" in row_data else ""
    # import_key based on
    # task
    # new_key = f"{payload.expected_start_date};{payload.expected_end_date};{payload.new_end_date}"
    # new_hash_key = hash_str_8_dig(new_key)
    # prev_hash_key, _, __ = split_str_get_key(input_data=payload.prev_hash_key, char_split = "--")
    # timesheet
    # new_key = f"{project_code};{parent_task};{employee_name};{progress};{activity_code};{task};{excel_task_status};{date_string}"
    # new_hash_key = hash_str_8_dig(new_key)
    # A_column_key = f"{new_hash_key}--{task_doc}--{new_time_sheet_doc.name}"
    # prev_hash_key, task_id, time_sheet_id = split_str_get_key(input_data=cell["A"], char_split="--")
    
    attrs["row_type"]            = row_data["B"] if "B" in row_data else ""
    attrs["project_code"]        = row_data["C"] if "C" in row_data else ""
    attrs["project_name"]        = row_data["D"] if "D" in row_data else ""

    attrs["expected_start_date"] = parser.parse(row_data["E"]) if "E" in row_data and row_data["E"] != "" else None
    attrs["expected_end_date"]   = parser.parse(row_data["F"]) if "F" in row_data and row_data["F"] != "" else None
    attrs["new_end_date"]        = parser.parse(row_data["G"]) if "G" in row_data and row_data["G"] != "" else None
    attrs["phase_name"]          = EXCEL_TYPE_PARENT_TASK[row_data["H"] if "H" in row_data else ""]
    # attrs["phase_name"           = EXCEL_TYPE_PARENT_TASK[row_data["H"] if "H" in row_data else ""]
    # expected_time = float(cell["I"]) if "I" in cell and cell["I"] != "" else 0.0
    attrs["expected_time"]       = float(row_data["I"]) if "I" in row_data and row_data["I"] != "" else 0.0
    # col_J: real time (sum)
    attrs["task_priority"]       = EXCEL_TASK_PRIORITY[row_data["K"] if "K" in row_data else ""]
    attrs["progress"]            = row_data["L"].replace("%", "") if "L" in row_data else ""
    attrs["employee_name"]       = row_data["M"] if "M" in row_data else ""
    attrs["activity_code"]       = row_data["N"] if "N" in row_data else ""
    attrs["task"]                = row_data["O"] if "O" in row_data else ""
    attrs["subject"]             = row_data["O"] if "O" in row_data else ""
    if attrs["subject"] == "":
        attrs["subject"] = "Other"
    attrs["excel_task_status"]   = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
    attrs["task_status"]         = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
    # self.task_number = num
    
    return attrs

In [9]:
def mapping_row_with_time_log(row_data, row_date_header):
    new_date = {}
    date_string = ""
    for column, value in row_data.items():
        if column in row_date_header and value != None and value != "":
            date = parser.parse(row_date_header[column])
            new_date[date] = value
            date_string = date_string + column + "-" + value + ";"
    return new_date, date_string

In [10]:
def hash_str_8_dig(raw_str):
    encode = hashlib.sha1(raw_str.encode("utf-8")).hexdigest()
    hash_obj = int(encode, 16) % (10 ** 8)
    return str(hash_obj)

In [11]:
cheap_hash = lambda input: hashlib.md5(input.encode("utf-8")).hexdigest()[:8]

In [13]:
# cheap_hash("pandion.vn")

In [12]:
# for caching
project_doc_list = {}
task_doc_list = {}
employee_doc_list = {}
timesheet_doc_list = {}

# for update sheet
update_records = {}

In [13]:
def is_row_empty(row_attrs):
    if row_attrs['project_code'] == "":
        return True
    return False

In [14]:
def is_row_timesheet_empty(row_attrs):
    if row_attrs['project_code'] == "":
        return True
    return False

In [15]:
def is_time_log_row_empty(row_time_logs):
    return True

In [16]:
def create_task():
    pass

In [17]:
def update_task():
    pass

In [18]:
def create_timesheet():
    pass

In [19]:
def update_timesheet():
    pass

In [20]:
def get_project(doc_name, project_code):
    # return if cached else get db & cache
    if project_doc_list[doc_name]:
        return project_doc_list[doc_name]
    else:
        doc = conn.get_doc('Project', doc_name)
        if doc:
            project_doc_list[doc_name] = doc
            return doc

    return None

In [21]:
def get_employee(doc_name, employee_name):
    # return if cached else get db & cache
    if employee_doc_list[doc_name]:
        return employee_doc_list[doc_name]
    else:
        doc = conn.get_doc('Employee', doc_name)
        if doc:
            employee_doc_list[doc_name] = doc
            return doc
    return None

In [22]:
def get_task(task_name):
    # return if cached else get db & cache
    if task_doc_list[doc_name]:
        return task_doc_list[doc_name]
    else:
        doc = conn.get_doc('Task', doc_name)
        if doc:
            task_doc_list[doc_name] = doc
            return doc
    return None

In [23]:
def get_timesheet(doc_name):
    # return if cached else get db & cache
    if timesheet_doc_list[doc_name]:
        return timesheet_doc_list[doc_name]
    else:
        doc = conn.get_doc('Timesheet', doc_name)
        if doc:
            timesheet_doc_list[doc_name] = doc
            return doc
    return None
    pass

In [62]:
def create_task(payload):
    # doc = { 
    #     "doctype": "Task"
    # }
    # doc["task_number"] = payload["task_number"]
    # doc["subject"] = payload["subject"]
    # doc["project"] = payload["project"]
    # doc["status"] = payload["status"]
    # doc["priority"] = payload["priority"]
    # doc["progress"] = payload["progress"]
    # doc["expected_time"] = payload["expected_time"]
    # doc["parent_task"] = payload["parent_task"]
    # doc["assigned_to"] = payload["employee_name"]
    # doc["company"] = payload["company"]
    conn.insert(payload)
    # pass

In [25]:
# def update_task(payload):
#     pass

In [27]:
def create_timesheet(doc_name, payload):

    # name = conn.get_value("Timesheet", "name", {"name": doc_name})
    # doc = conn.get_doc("Timesheet", doc_name)
    # if doc is None:
    doc = { 
        "doctype": "Timehsheet",
        "naming_series": "TS-.YYYY.-",
    }

    doc["parent_project"] = payload["project_code"]
    doc["employee"] = payload["emp_name"]
    doc["status"] = payload["ts_status"]
    doc["company"] = payload["company"]
    doc["time_logs"] = payload["time_logs"]

    conn.insert(doc)
    # if len(dates) > 0 and activity_code != "":
    #     for date, hrs in dates.items():
    #         time_sheet_doc.append(
    #             "time_logs",
    #             {
    #                 "activity_type": activity_code,
    #                 "from_time": date,
    #                 "hours": flt(hrs),
    #                 "project": project_code,
    #                 "task": task_doc,
    #                 "completed": excel_task_status == "Done",
    #             },
    #         )
    # if ts_status == "Submitted":
    #     time_sheet_doc.submit()
    # elif ts_status == "Cancelled":
    #     time_sheet_doc.insert()
    #     frappe.db.set_value("Timesheet", time_sheet_doc.name, {
    #         "status": "Cancelled",
    #         "docstatus": 2
    #     })
    # else:
    #     time_sheet_doc.insert()
    # pass

In [28]:
# def update_timesheet(name, payload):
#     pass

In [74]:
# import_key for task: task_id||
def handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company):

    print(f"  Handle parent task at row: {idx}\n"
          f"  import_key: {row_attrs['import_key']}\n"
          f"  project_code: {row_attrs['project_code']}\n"
          f"  project_code: {row_attrs['phase_name']}\n"
          f"  subject: {row_attrs['subject']}\n"
          f"  task: {row_attrs['task']}")

    doc_name = conn.get_value("Task", 
                              ["name"],
                              [["subject", "=", row_attrs['subject']],
                               ["project", "=", row_attrs['project_code']], 
                               ["type", "=", row_attrs['phase_name']],
                               ["is_group", "=", 1]])
    if doc_name:
        print(f"doc_name existed")
        # existed
        pass
    else:
        # create new
        payload = {}
        payload["doctype"] = "Task"
        payload["subject"] = row_attrs["subject"]
        payload["project"] = row_attrs["project_code"]
        payload["type"] = row_attrs["phase_name"]
        payload["is_group"] = 1
        print(f"{payload}")
        create_task(payload)
    
    if row_attrs['import_key'] != "":
        imp_keys = row_attrs['import_key'].split('||')
        if len(imp_keys) < 2:
            # wrong key -> action like new parent
            # handle new parent task
            # create_task(payload = payload)
            pass
        else:
            # handle update parent task
            # check import_key with parent_task['name']
            pass 
    else:
        # handle new parent task
        # create_task(payload = payload)
        pass
        
    # print(f"  Handle parent task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  project_code: {row_attrs['phase_name']}\n"
    #       f"  subject: {row_attrs['subject']}\n"
    #       f"  task: {row_attrs['task']}")

In [30]:
# import_key for task: timesheet_id||
def handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, parent_task, company):
    time_logs_date, time_logsdate_string = mapping_row_with_time_log(row_data, row_date_header)
    if row_attrs['import_key'] != "":
        # handle update timesheet + task
        pass
    else:
        # handle new timesheet + task
        pass
    
    print(f"  Handle task at row: {idx}\n"
          f"  import_key: {row_attrs['import_key']}\n"
          f"  project_code: {row_attrs['project_code']}\n"
          f"  project_code: {row_attrs['phase_name']}\n"
          f"  subject: {row_attrs['subject']}\n"
          f"  employee_name: {row_attrs['employee_name']}\n"
          f"  task: {row_attrs['task']}\n"
          f"  parent task: {parent_task}\n"
          f"  time_logs: {time_logs_date}")

In [31]:
current_parent_task_row = 0
current_parent_task_doc = {}

In [68]:
# if row is empty
    # skip this row
# if col_B == "P"
    # handle as parent task
    # if col_A is not empty (parent task is created)
        # handle update parent task if any changes (compare hash)
    # else
        # handle as new parent task
# else
    # handle as task with timesheet
    # if col_A is not empty (timesheet is created)
        # handle update timesheet if any changes (compare hash)
    # else
        # handle as new timesheet
def handle_timesheet_row(idx, row_data, row_date_header, company):
    global current_parent_task_row
    global current_parent_task_doc
    row_attrs = mapping_row_with_attr(row_data)
    if is_row_empty(row_attrs):
        return
    
    if row_attrs['row_type'] == "P":
        # handle as parent task
        if row_attrs['project_code'] in ["HR01_ACO.00"]:
            return
        handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company)
        current_parent_task_row = idx
        current_parent_task_doc = {}
    else:
        # handle as task with timesheet
        # handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, current_parent_task_row, company)
        pass

In [48]:
def handle_timesheet_sheet():
    try:
        # row_date_header = await get_row_date_header()
        # all_records = await ggSheet.get_all_row_values()
        
        current_parent_task_row = 0
        for idx, row_data in enumerate(all_records):
            if idx >= row_start and idx <= row_end:
                print(f"idx: {idx}")
                # print(row_data)
                handle_timesheet_row(idx, row_data, row_date_header, company)
                # break
    except Exception as err:
        print(f"Handle timesheet {worksheet_name} failed with: {err}")

In [34]:
url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
worksheet_name="BAP"

In [35]:
ggSheet = GGSheet(url_file, worksheet_name)

In [36]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)

In [37]:
all_records = await ggSheet.get_all_row_values()

In [79]:
handle_timesheet_sheet()

idx: 5
idx: 6
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
idx: 13
  Handle parent task at row: 13
  import_key: 76282495--866740a90c
  project_code: U00123_BAP.00
  project_code: Other
  subject: Other
  task: 
doc_name existed
  Handle parent task at row: 13
  import_key: 76282495--866740a90c
  project_code: U00123_BAP.00
  project_code: Other
  subject: Other
  task: 
idx: 14
idx: 15
idx: 16
  Handle parent task at row: 16
  import_key: 76282495--20fddd17b2
  project_code: U00223_BAP.00
  project_code: Other
  subject: Other
  task: 
doc_name existed
  Handle parent task at row: 16
  import_key: 76282495--20fddd17b2
  project_code: U00223_BAP.00
  project_code: Other
  subject: Other
  task: 
idx: 17
idx: 18
  Handle parent task at row: 18
  import_key: 76282495--a70a24c2ae
  project_code: U00323_BAP.00
  project_code: Other
  subject: Other
  task: 
doc_name existed
  Handle parent task at row: 18
  import_key: 76282495--a70a24c2ae
  project_code: U00323_BAP.00
  project_code: Othe

In [80]:
url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
worksheet_name="T2"

In [81]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [82]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [85]:
handle_timesheet_sheet()

idx: 5
  Handle parent task at row: 5
  import_key: 7590380--dm4ammu9aa
  project_code: P33422_HTP.00
  project_code: Construction
  subject: THIẾT KẾ THI CÔNG
  task: THIẾT KẾ THI CÔNG
doc_name existed
  Handle parent task at row: 5
  import_key: 7590380--dm4ammu9aa
  project_code: P33422_HTP.00
  project_code: Construction
  subject: THIẾT KẾ THI CÔNG
  task: THIẾT KẾ THI CÔNG
idx: 6
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
idx: 13
idx: 14
idx: 15
idx: 16
idx: 17
idx: 18
idx: 19
idx: 20
  Handle parent task at row: 20
  import_key: 76282495--dm4a9rnsgf
  project_code: P33422_HTP.00
  project_code: Other
  subject: MOCKUP VILLAS
  task: MOCKUP VILLAS
doc_name existed
  Handle parent task at row: 20
  import_key: 76282495--dm4a9rnsgf
  project_code: P33422_HTP.00
  project_code: Other
  subject: MOCKUP VILLAS
  task: MOCKUP VILLAS
idx: 21
idx: 22
idx: 23
idx: 24
idx: 25
idx: 26
idx: 27
idx: 28
idx: 29
idx: 30
idx: 31
idx: 32
  Handle parent task at row: 32
  import_key: 76282495--d